<h1> A liga ternária mais cara do mundo</h1>

<h2> Introdução </h2>

<p style="text-align: justify">As ligas ternárias são compostas por três elementos químicos diferentes, combinados em proporções definidas. Essas ligas desempenham um papel fundamental em diversas áreas, como a indústria, a pesquisa científica e a tecnologia. A combinação de três elementos permite a criação de materiais com propriedades e características únicas, que podem ser personalizadas de acordo com as necessidades específicas de cada aplicação e são frequentemente exploradas em áreas como a nanotecnologia, a ciência dos materiais e a engenharia de superfícies.</p>

<p style="text-align: justify">Essas ligas são frequentemente utilizadas em setores como a indústria automobilística, aeroespacial, eletrônica, metalúrgica e biomédica. Na indústria automobilística, por exemplo, as ligas ternárias podem ser usadas na fabricação de componentes leves e resistentes, contribuindo para a redução do peso dos veículos e melhorando a eficiência energética.</p>

<p style="text-align: justify">Além disso, as ligas ternárias também desempenham um papel importante na pesquisa científica, permitindo o estudo e o desenvolvimento de novos materiais com propriedades avançadas. Essas ligas são frequentemente exploradas em áreas como a nanotecnologia, a ciência dos materiais e a engenharia de superfícies.</p>

<h2> Objetivo </h2>

<p style="text-align: justify">Encontrar uma liga de três elementos que tenha o maior custo possível. A liga ternária deve ser da forma $x\text{A}y\text{B}z\text{C}$ sendo que $x+y+z = 100\,\mathrm{g}$, $x>5\,\mathrm{g}$, $y>5\,\mathrm{g}$, $z>5\,\mathrm{g}$ e &ldquo;A&rdquo;, &ldquo;B&rdquo; e &ldquo;C&rdquo; são elementos químicos. Considerando que qualquer composto com 3 elementos químicos é chamado de liga.</p>

<h3> Desenvolvimento do algortimo </h3>

<p style="text-align: justify"> Para resolver este problema, é necessário pensar primeiramente no formato dos individuos que serão gerados e mutados. Para facilitar a iteração, cada atributo será trabalhado enquanto listas, afim de manipulá-los individualmente.</p>
 
<h4> Parametros iniciais </h4>
<ul>
    <li>A liga deve pesar 100g </li>
    <li>Os pesos e valores são determinados por uma lista pré-existente</li>
    <li>A parcela de cada elemento na liga deve ser maior que 5%</li>
</ul>

<h4> Passos</h4>
<ul>
    <li> Selecionar genes, que serão elementos da lista</li>
    <li> Gerar individuo válido, combinando cada elemento da liga e seu respectivo preço com uma massa </li>
    <li> Gerar uma população </li>
    <li> Avaliar seu respectivo preço (fitness_ind e fitness_pop) </li>
    <li> Realizar cruzamento entre os individuos da população </li>

## Importações



Todos os comandos de `import` devem estar dentro desta seção.



In [1]:
import random

from funcoes import funcao_obj_pop_liga
from funcoes import funcao_obj_liga
from funcoes import populacao_liga as cria_populacao_inicial
from funcoes import computa_massa_pop
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_liga as funcao_cruzamento
from funcoes import mutacao_liga as funcao_mutacao

## Códigos e discussão



<p style="text-align: justify"> Antes de iniciar o algoritmo, é necessario declarar as constates envolvidas, tanto as relacionadas aos processos do algoritmo, quanto aquelas específicas deste problema. Assim, declaramos além do número de genes, tamanho da população, número de gerações, chance de cruzamento e chance de mutação, que já são caracterísitcos de algoritmos genéticos, temos também um dicionário onde as chaves são os nomes dos elementos e os valores são seus respectivos valores por kilograma, um segundo dicionário, similar ao primeiro, mas com os valores convertidos para gramas. </p>

In [2]:
### CONSTANTES

# relacionadas à busca
NUM_GENES = 3 # número de elementos em cada individuo
TAMANHO_POP = 92 # tamanho da população
NUM_GERACOES = 100 # número de gerações 
CHANCE_CRUZAMENTO = 0.5 
CHANCE_MUTACAO = 0.05

# relacionadas ao problema 
# preço em dólares por kilograma
ELEMENTOS = {
    "H": 1.39,
    "He": 24,
    "Li": 85.6,
    "Be": 857,
    "B": 3.68,
    "C": 0.122,
    "N": 0.14,
    "O": 0.154,
    "F": 2.16,
    "Ne": 240,
    "Na": 3.43,
    "Mg": 2.32,
    "Al": 1.79,
    "Si": 1.7,
    "P": 2.69,
    "S": 0.0926,
    "Cl": 0.082,
    "Ar": 0.931,
    "K": 13.6,
    "Ca": 2.35,
    "Sc": 3460,
    "Ti": 11.7,
    "V": 385,
    "Cr": 9.4,
    "Mn": 1.82,
    "Fe": 0.424,
    "Co": 32.8,
    "Ni": 13.9,
    "Cu": 6,
    "Zn": 2.55,
    "Ga": 148,
    "Ge": 1010,
    "As": 1.31,
    "Se": 21.4,
    "Br": 4.39,
    "Kr": 290,
    "Rb": 15500,
    "Sr": 6.68,
    "Y": 31,
    "Nb": 85.6,
    "Mo": 40.1,
    "Tc": 100000,
    "Ru": 10600,
    "Rh": 147000,
    "Pd": 49500,
    "Ag": 521,
    "Cd": 2.73,
    "In": 167,
    "Sn": 18.7,
    "Sb": 5.79,
    "Te": 63.5,
    "I": 35,
    "Xe": 1800,
    "Cs": 61800,
    "Ba": 0.275,
    "La": 4.92,
    "Ce": 4.71,
    "Pr": 103,
    "Nd": 57.5,
    "Pm": 460000,
    "Sm": 13.9,
    "Eu": 31.4,
    "Gd": 28.6,
    "Tb": 658,
    "Dy": 307,
    "Ho": 57.1,
    "Er": 26.4,
    "Tm": 3000,
    "Yb": 17.1,
    "Lu": 643,
    "Hf": 900,
    "Ta": 312,
    "W": 35.3,
    "Re": 4150,
    "Os": 12000,
    "Ir": 56200,
    "Pt": 27800,
    "Hg": 30.2,
    "Tl": 4200,
    "Pb": 2,
    "Bi": 6.36,
    "Po": 49200000000000,
    "Ac": 29000000000000,
    "Th": 287,
    "Pa": 280000,
    "U": 101,
    "Np": 660000,
    "Pu": 6490000,
    "Am": 750000,
    "Cm": 160000000000,
    "Bk": 185000000000,
    "Cf": 185000000000,
}

# convertendo valores para gramas
PRECO_POR_GRAMA = {}
for i in ELEMENTOS:
    a = ELEMENTOS[i]/1000
    PRECO_POR_GRAMA[i] = a


<p style="text-align: justify">Para resolver este problema, é preciso aplicar restrições ao algoritmo, de forma que ele obedeça aos parâmetros postos. Desta forma, cria-se primeiro indivíduos válidos. O problema fornece um dicionário onde as chaves são os nomes dos elementos e os valores correspondem preço por kilograma, neste caso, é conveniente converter a unidade para gramas, a unidade que será trabalhada.</p>
 
<p style="text-align: justify">Para que o indivíduo seja válido, a soma das massas de cada elemento presente na liga deve ser igual a 100g. Esses valores serão importantes para calcular o valor final da liga, pois os preços, como já mencionado, são dados em unidade/g. Desta forma, a função <mark style="text-align: center; background-color: #363636; color: white; font-family: courier; ">  computa_massas</mark> (funcoes.py) gera uma lista com três valores entre 5 e 90 que serão relacionados a cada elemento da lista quando necessario. Estes valores não sofrem alteração durante o cruzamento ou mutação, pois isso pode comprometer a soma e quebrar a restrição de massa final. </p>

<p style="text-align: justify"> Outro ponto importante para considerar o individuo válido, é que ele seja composto por três elementos diferentes, e que essa propriedade se mantenha mesmo após as etapas de cruzamento e mutação. Para isso, foi criada uma função <mark style="text-align: center; background-color: #363636; color: white; font-family: courier"> checa_individuo</mark>, que checa se há ou não elementos repetidos, está função foi encrementada na geração do individuo e na mutação. Para a função que realiza o cruzamento, foi utilizado um método diferente, já que os únicos genes disponíveis para gerar novos individuos são os do pai e da mãe, logo, foi necessário análisar os genes disponíveis para que não se repitam dentro da função</p>

<p style="text-align: justify"> O fitnesse de cada individou é avaliado de acordo com o produto da massa pelo preço (por grama) de cada elemento, a soma dos produtos corresponde ao valor total da liga em unidades de dinheiro. Sendo este um problema de maximização, o criterio de seleção é o individuo com maior fitness.</p>

In [3]:
# BUSCA POR ALGORITMO GENÉTICO 

# gerando a população inicial 
populacao = cria_populacao_inicial(NUM_GENES, PRECO_POR_GRAMA, TAMANHO_POP)
populacao_massas = computa_massa_pop(NUM_GENES, TAMANHO_POP)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf") # Como este é um problema de maximização, começamos com um fitness
                                        # extremamente baixo, assim qualquer valor acima disto será considerado
melhor_individuo_ja_visto = [0] * (len(ELEMENTOS)) 

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_obj_pop_liga(populacao, populacao_massas)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    
    # dividindo a população pela metade
    pais = populacao[0::2]   
    maes = populacao[1::2]

    contador = 0

    # relacionado as duas metades da população
    for pai, mae in zip(pais, maes):
                
        if random.random() <= CHANCE_CRUZAMENTO:  # determina se vai ou não haver cruzamento 
            filho1, filho2 = funcao_cruzamento(pai, mae)
            
            # substituindo os novos individuos na população
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2
        
    
    # Mutação
    # Altera um gene aleatório dos individuos 
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO: # determina se vai ou não haver mutação
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo, PRECO_POR_GRAMA)
            

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_obj_pop_liga(populacao, populacao_massas) # fitness de todos os individuos da população
    maior_fitness = max(fitness) # maior valor de fitness encontrado
    
    posicao = fitness.index(maior_fitness) # identificando em que posição da população está o melhor fitness
    individuo = populacao[posicao].copy() # relacionando o individuo e o melhor fitness pela posição
    valor = funcao_obj_liga(individuo) # identificando qual é o valor do melhor fitness
    
    if maior_fitness > melhor_fitness_ja_visto: # sempre que o algoritmo encotrar um valor maior de fitness
                                                # ele será armazernado na variável 'melhor_fitness_ja_visto' 
                                                # até que haja alguém melhor
        melhor_fitness_ja_visto = maior_fitness 
        melhor_individuo_ja_visto = individuo   
        
        # para visualizar a liga de forma limpa
        posicao_elemento = 0
        liga = individuo.copy()
        for elemento in liga:
            nome_elemento = elemento[0]
            liga[posicao_elemento] = nome_elemento
            posicao_elemento += 1
        
        print(f"Maior valor: {valor}, para a liga {liga}")


# reportando o melhor individuo encontrado
print()
melhor_liga = funcao_obj_liga(melhor_individuo_ja_visto)

print(
    f"Com isso, a liga ternária mais cara encontra foi {liga}, "
    f"com o valor de {melhor_liga} dinheiros "
)

Maior valor: 2432000000000.007, para a liga ['Ac', 'Po', 'Ru']
Maior valor: 1426800000120.41, para a liga ['Te', 'Po', 'Ru']
Maior valor: 1399800000003.683, para a liga ['Ir', 'Ru', 'Nb']
Maior valor: 2585800000402.8, para a liga ['Am', 'Ac', 'O']
Maior valor: 3257000000275.6, para a liga ['Ru', 'O', 'Nb']
Maior valor: 4273600000000.0, para a liga ['Ru', 'O', 'Po']
Maior valor: 4637200000000.0, para a liga ['Ru', 'O', 'Po']

Com isso, a liga ternária mais cara encontra foi ['Ru', 'O', 'Po'], com o valor de 3566600000000.0 dinheiros 


## Conclusão



Delete este texto e escreva sua conclusão.



## Referências consultadas



1.  Delete este texto e inclua suas referências ordenadas numericamente. Se for referenciar no notebook, use o número entre colchetes (exemplo: para citar essa referência aqui escreva &ldquo;[1]&rdquo; sem as áspas).

2.  Cada item deve ser numerado. Siga o padrão apresentado.

3.  Caso não tenha nenhuma referência consultada, delete esta seção e o texto contido nela!



## Playground

